In [1]:
import pandas as pd
from pandas_datareader import data
import matplotlib.pyplot as plt
import yfinance as yf
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
# pyfolio  qgrid empyrical
import pickle
import numpy as np
import time
import seaborn as sns
from itertools import combinations
import multiprocessing as mp
from tqdm import tqdm
import sys
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import os
current_path = os.getcwd()

In [3]:
sys.path.append('C:\\Users\\Rafael\\Trading\\Trading')

In [4]:
import TradingModule as td

In [5]:
with open('C:\\Users\\Rafael\\Trading\\Trading\\ibov.pkl','rb') as file:
     data =pickle.load(file)
data1 =data['Adj Close']

In [6]:
import parallel as pl

In [7]:
print(pl.parallelize(data1,rollmeanstrat))

NameError: name 'rollmeanstrat' is not defined

In [8]:
pd.options.mode.chained_assignment = None

In [11]:
inic =''
def estrat(x):
    global inic
    if inic!=x:
        inic = x
        return x
    
    else:
        return np.nan


def rollmeanstrat(serie,longmean=100,shortmean=10):
    start_time = time.clock()

    serie =pd.DataFrame(serie)

    print(serie)



    serie['roll']   =  serie.rolling(shortmean,min_periods=5).mean()-serie.rolling(longmean,min_periods=5).mean()
    

    serie['DECISAO']=np.nan
    
    serie.loc[serie['roll']>0.0,'DECISAO']='COMPRA'
    serie.loc[serie['roll']<=0.0,'DECISAO'] ='VENDA'
    
    serie['ESTRAT'] =serie['DECISAO'].apply(func=estrat)

    
    x =returns(serie)
    
    return x
def returns(serie,tipo_oper ='COMPRA'):
    start_time = time.clock()
    
    serie=serie.dropna()
    
    serie['LUCRO']= serie.iloc[:,0].shift(-1)/(serie.iloc[:,0])
   
    
    serie= serie.loc[(serie['ESTRAT']==tipo_oper) & (pd.notna(serie['LUCRO']))]
    
    
   
    
   
    retorno_total=serie['LUCRO'].rolling(len(serie )).apply(np.prod, raw=True).dropna()
   
    return retorno_total

In [12]:
start_time = time.clock()
print(rollmeanstrat(data1['BBAS3.SA'],100,50))
print (1,time.clock() - start_time, "seconds")

             BBAS3.SA
Date                 
2014-12-22  18.839794
2014-12-23  18.770786
2014-12-24        NaN
2014-12-25        NaN
2014-12-26  18.824457
...               ...
2019-12-16  47.869999
2019-12-17  48.950001
2019-12-18  50.639999
2019-12-19  51.160000
2019-12-20  50.459999

[1305 rows x 1 columns]
Date
2019-06-26    1.116868
Name: LUCRO, dtype: float64
1 0.032335500000044703 seconds


In [ ]:
start_time = time.clock()
test =data1.apply(rollmeanstrat,args=(50,9))
print (time.clock() - start_time, "seconds")

In [47]:
params =np.array_split([i for i in combinations(range(10,100,5),2)], mp.cpu_count() - 1)

In [48]:
start_time = time.clock()
if __name__ == '__main__':
    pool = mp.Pool(processes = (mp.cpu_count()))
    results = pool.map(td.worker, params)
    
    pool.close()
    pool.join()
print (time.clock() - start_time, "seconds")

30.429786799999988 seconds


In [55]:
results[10]

{(70, 85): 1.5754897592631398,
 (70, 90): 1.6147184294337018,
 (70, 95): 1.6108954357758194,
 (75, 80): 1.5075272903164167,
 (75, 85): 1.5614178098051918,
 (75, 90): 1.5796124773009896,
 (75, 95): 1.6100459230747568,
 (80, 85): 1.726666157710107,
 (80, 90): 1.5974075325893589,
 (80, 95): 1.5565307141351372,
 (85, 90): 1.5734788603129757,
 (85, 95): 1.5638476126556873,
 (90, 95): 1.5701709863301503}

In [62]:
dic ={}

In [ ]:
[dic.update(i) for i in results]

In [69]:
pd.Series(dic).drop_duplicates()

10  15    1.410361
    20    1.248606
    25    1.230880
    30    1.187015
    35    1.155820
            ...   
80  90    1.597408
    95    1.556531
85  90    1.573479
    95    1.563848
90  95    1.570171
Length: 153, dtype: float64

In [14]:
start_time = time.clock()
pars = {}
pars2 = {}

for z in tqdm(range(10,50,5), position=0, leave=True):
    lista =[] 
    for y in range(10,50,5):
        
        if y<z:
            print(z,y)
            test =data1.apply(rollmeanstrat,args=(z,y)) 
            
            pars[(z,y)]=test.sum().mean()
            pars2[(z,y)]=test.sum().median()
print (time.clock() - start_time, "seconds")

  0%|          | 0/8 [00:00<?, ?it/s]

15 10
             ABEV3.SA
Date                 
2014-12-22  13.898333
2014-12-23  13.838793
2014-12-24        NaN
2014-12-25        NaN
2014-12-26  13.796264
...               ...
2019-12-16  18.591703
2019-12-17  18.426054
2019-12-18  18.601448
2019-12-19  18.679399
2019-12-20  18.959999

[1305 rows x 1 columns]
             ABEV3.SA
Date                 
2014-12-22  13.898333
2014-12-23  13.838793
2014-12-24        NaN
2014-12-25        NaN
2014-12-26  13.796264
...               ...
2019-12-16  18.591703
2019-12-17  18.426054
2019-12-18  18.601448
2019-12-19  18.679399
2019-12-20  18.959999

[1305 rows x 1 columns]
             AZUL4.SA
Date                 
2014-12-22        NaN
2014-12-23        NaN
2014-12-24        NaN
2014-12-25        NaN
2014-12-26        NaN
...               ...
2019-12-16  57.270000
2019-12-17  56.869999
2019-12-18  57.610001
2019-12-19  57.709999
2019-12-20  56.759998

[1305 rows x 1 columns]
             B3SA3.SA
Date                 
2014-12-22   8.19

C:\Users\Rafael\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)



             SBSP3.SA
Date                 
2014-12-22  15.246716
2014-12-23  15.353088
2014-12-24        NaN
2014-12-25        NaN
2014-12-26  15.432865
...               ...
2019-12-16  58.540001
2019-12-17  58.130001
2019-12-18  59.310001
2019-12-19  59.639999
2019-12-20  59.700001

[1305 rows x 1 columns]
             SMLS3.SA
Date                 
2014-12-22  31.434816
2014-12-23  31.791348
2014-12-24        NaN
2014-12-25        NaN
2014-12-26  31.617624
...               ...
2019-12-16  38.500000
2019-12-17  38.450001
2019-12-18  38.880001
2019-12-19  39.209999
2019-12-20  38.790001

[1305 rows x 1 columns]
                   SPY
Date                  
2014-12-22  188.007996
2014-12-23  188.261719
2014-12-24  188.279846
2014-12-25         NaN
2014-12-26  188.886978
...                ...
2019-12-16  317.936859
2019-12-17  318.006531
2019-12-18  318.026398
2019-12-19  319.329987
2019-12-20  320.730011

[1305 rows x 1 columns]
             SUZB3.SA
Date                 
2014-12-2

 25%|██▌       | 2/8 [00:01<00:03,  1.65it/s]


             YDUQ3.SA
Date                 
2014-12-22        NaN
2014-12-23        NaN
2014-12-24        NaN
2014-12-25        NaN
2014-12-26        NaN
...               ...
2019-12-16  47.349998
2019-12-17  46.349998
2019-12-18  48.139999
2019-12-19  49.700001
2019-12-20  48.990002

[1305 rows x 1 columns]
               ^BVSP
Date                
2014-12-22   50121.0
2014-12-23   50890.0
2014-12-24       NaN
2014-12-25       NaN
2014-12-26   50145.0
...              ...
2019-12-16  111896.0
2019-12-17  112616.0
2019-12-18  114315.0
2019-12-19  115131.0
2019-12-20  115121.0

[1305 rows x 1 columns]
                   ^N225
Date                    
2014-12-22  17635.140625
2014-12-23           NaN
2014-12-24  17854.230469
2014-12-25  17808.750000
2014-12-26  17818.960938
...                  ...
2019-12-16  23952.349609
2019-12-17  24066.119141
2019-12-18  23934.429688
2019-12-19  23864.849609
2019-12-20  23816.630859

[1305 rows x 1 columns]
20 10
             ABEV3.SA
Date        

 38%|███▊      | 3/8 [00:03<00:05,  1.14s/it]


             UGPA3.SA
Date                 
2014-12-22  20.399210
2014-12-23  20.391304
2014-12-24        NaN
2014-12-25        NaN
2014-12-26  20.252935
...               ...
2019-12-16  22.430000
2019-12-17  23.000000
2019-12-18  23.690001
2019-12-19  24.290001
2019-12-20  24.490000

[1305 rows x 1 columns]
            USIM5.SA
Date                
2014-12-22  5.137269
2014-12-23  5.195757
2014-12-24       NaN
2014-12-25       NaN
2014-12-26  5.020291
...              ...
2019-12-16  9.120000
2019-12-17  9.250000
2019-12-18  9.250000
2019-12-19  9.320000
2019-12-20  9.330000

[1305 rows x 1 columns]
             VALE3.SA
Date                 
2014-12-22  18.549894
2014-12-23  19.287720
2014-12-24        NaN
2014-12-25        NaN
2014-12-26  18.818983
...               ...
2019-12-16  53.340000
2019-12-17  53.419998
2019-12-18  53.470001
2019-12-19  54.000000
2019-12-20  54.700001

[1305 rows x 1 columns]
             VIVT4.SA
Date                 
2014-12-22  31.935505
2014-12-23  3

 38%|███▊      | 3/8 [00:05<00:08,  1.77s/it]


KeyboardInterrupt: 

In [ ]:
heat =pd.Series(pars).sort_values(ascending=False)
pd.Series(pars).sort_values(ascending=False)

In [ ]:
heat =heat.reset_index()

In [ ]:
sns.heatmap(heat.pivot('level_0','level_1',0))

In [ ]:
heat =pd.Series(pars2).sort_values(ascending=False)
heat =heat.reset_index()
sns.heatmap(heat.pivot('level_0','level_1',0))